In [1]:
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score,recall_score
from imblearn.over_sampling import SMOTE
from IPython.display import SVG,display
import numpy as np
import preprocessor as p
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases#from sklearn import model_selection, naive_bayes, svm
from sklearn.naive_bayes import GaussianNB
from yellowbrick.classifier import DiscriminationThreshold
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

In [2]:
%run ./Utils.ipynb

In [3]:
np.random.seed(500)

In [4]:
data = pd.read_csv('data/output.csv', sep='\t', index_col=False)

In [5]:
data = data.drop_duplicates(keep='first')
data = data.dropna()
data['Sentiment'] = data['Sentiment'].astype(int)

In [30]:
data['Sentiment'].value_counts()

-1    4591
 1    2944
 0    2296
Name: Sentiment, dtype: int64

In [31]:
4591+2944+2296

9831

In [6]:
pos_words = []
pos_lable = []
with open('data/SinPos1.txt','r',encoding = 'UTF-8') as f:
    for line in f:
        line = line.strip()
        if line != '':
            pos_words.append(line)
            pos_lable.append(1)

In [7]:
neg_words = []
neg_lable = []
with open('data/SinNeg1.txt','r',encoding = 'UTF-8') as f:
    for line in f:
        line = line.strip()
        #print(line)
        #neg_words.append(line)
        if line != '':
            neg_words.append(line)
            neg_lable.append(-1)

In [8]:
Tweets = pos_words + neg_words
Sentiment = pos_lable + neg_lable

In [9]:
data1 = {'Tweet':Tweets, 'Sentiment':Sentiment}

In [10]:
data_new = pd.DataFrame(data1)

In [11]:
final_data = pd.concat([data_new, data],ignore_index=True) 

In [12]:
# Cleaning data - remove punctuation from every newsgroup text
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.HASHTAG)

cleaned_tweets = []
for line in final_data['Tweet']:
    line = line.strip()
    line = p.clean(line)
    line = re.sub(r'[\!"#$%&\*+,-./:;<=>?@^_`()|~=]','',line).strip().split(' ')
    cleaned_tweets.append(line)

In [13]:
word2vec_file = 'word2vec_new_301119' + str(200) + '.model'
model = Word2Vec.load(word2vec_file)

In [14]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.wv.vocab]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

In [15]:
sentence_vectors = []
for doc in cleaned_tweets:
    vec = get_mean_vector(model, doc)
    if len(vec) > 0:
        sentence_vectors.append(vec)
    else:
        sentence_vectors.append(0)

In [16]:
final_df = pd.DataFrame(sentence_vectors) 

In [17]:
final_df.columns = final_df.columns.astype(str)

In [18]:
sentiment = final_data["Sentiment"]

In [19]:
final_data["Sentiment"].value_counts()

-1    15645
 1     8045
 0     2296
Name: Sentiment, dtype: int64

In [20]:
train_x = final_df[:22986]
test_x = final_df[22986:]
train_y = sentiment[:22986]
test_y = sentiment[22986:]

In [21]:
test_y.value_counts()

-1    1271
 0     891
 1     838
Name: Sentiment, dtype: int64

In [22]:
len(train_x),len(test_x),len(train_y),len(test_y)

(22986, 3000, 22986, 3000)

In [26]:
def prediction_alg(algorithm,training_x,testing_x,
                                 training_y,testing_y,threshold_plot = True) :
    
    #model
    algorithm.fit(training_x,training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    
    print (algorithm)
    print ("\n Classification report : \n",classification_report(testing_y,predictions))
    print ("Accuracy Score   : ",accuracy_score(testing_y,predictions))
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y,predictions)
    
    
#     if threshold_plot == True : 
#         visualizer = DiscriminationThreshold(algorithm)
#         visualizer.fit(training_x,training_y)
#         visualizer.poof()

In [27]:
lgbm_c = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                        learning_rate=0.5, max_depth=7, min_child_samples=20,
                        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
                        n_jobs=-1, num_leaves=500, objective='multiclass', random_state=None,
                        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                        subsample_for_bin=200000, subsample_freq=0, metric = 'multi_logloss', num_class = 3,)

prediction_alg(lgbm_c,train_x,test_x,train_y,test_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.5, max_depth=7,
               metric='multi_logloss', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_class=3, num_leaves=500, objective='multiclass',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

 Classification report : 
               precision    recall  f1-score   support

          -1       0.50      0.79      0.61      1271
           0       0.54      0.13      0.21       891
           1       0.40      0.37      0.39       838

    accuracy                           0.48      3000
   macro avg       0.48      0.43      0.40      3000
weighted avg       0.48      0.48      0.43      3000

Accuracy Score   :  0.47633333333333333


In [28]:
xgc = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                    colsample_bytree=1, gamma=0, learning_rate=0.9, max_delta_step=0,
                    max_depth = 7, min_child_weight=1, missing=None, n_estimators=100,
                    n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
                    reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                    silent=True, subsample=1,num_classes = 3)

prediction_alg(xgc,train_x,test_x,train_y,test_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.9, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=1, nthread=1, num_classes=3,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=True, subsample=1, tree_method=None,
              validate_parameters=False, ...)

 Classification report : 
               precision    recall  f1-score   support

          -1       0.49      0.77      0.60      1271
           0       0.48      0.14      0.22       891
           1       0.41      0.37      0.39       838

    accuracy                           0.47      3000
   macro avg       0.46      0.

In [29]:
adb = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=200
)

prediction_alg(adb,train_x,test_x,train_y,test_y)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=2,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          